# Grafote

### Qué va a analizar nuestro grafo?

Analiza las notas de final hacia los estudiantes

### Qué queremos responder?

¿Cuáles son las materias más justas? ¿Quiénes son los mejores estudiantes? ¿Cuáles fueron las notas de final más injustas?

### Cómo es el grafo?

- Nodos: Materias por un lado, estudiantes por el otro
- Aristas: Un estudiante terminó de cursar una materia, la arista representa la nota y está direccionada de la materia hacia el estudiante (materia valoró a este estudiante con esta nota)
- Peso: Nota de final

## Preprocesamiento

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
from tqdm import tqdm

In [ ]:
df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

# join con el plan, para tenerlo
df = pd.merge(df, pd.read_csv('informatica.csv'))

# filtros de columnas relevantes
df = df[['Padron', 'materia_id', 'materia_nota', 'materia_nombre']]
df.head()

In [ ]:
# Se toman solo cursadas aprobadas
df_notas = df[df['materia_nota'] >= 4].copy()

df_notas['nota_mediana'] = df_notas.groupby('Padron')['materia_nota'].transform('median')
df_notas['nota_promedio'] = df_notas.groupby('Padron')['materia_nota'].transform('mean')
# Para reducir ruido, se eliminan mapas donde "casi" todas las notas sean 4
df_notas = df_notas[df_notas['nota_mediana'] > 4.5]

df_notas.shape

In [ ]:
# Normalizar notas de [-1 a 1]
df_notas['materia_nota_norm'] = (df_notas['materia_nota'] - 7) / 3
df_notas['materia_nota_norm'].value_counts()

In [ ]:
# Se agregan columnas de cantidades, para luego filtrar más adelante
df_notas['cant_materias'] = df_notas.groupby(['Padron'])['materia_id'].transform('count')
df_notas['cant_padrones'] = df_notas.groupby(['materia_id'])['Padron'].transform('count')

## Algoritmo REV2

In [ ]:
def rev2(
    df,
    col_in,
    col_out,
    col_rating,
    gamma_1=0.5,
    gamma_2=0.5,
    n_iteraciones=15,
):
    """Algoritmo REV2. Dado un dataframe con entradas, salidas y ratings
    denotados por los nombres de columna, devuelve series para aplicar los
    valores de F, G y R.
    """
    assert df[col_rating].min() == -1 and df[col_rating].max() == 1, "Rating no fue normalizado a [-1, 1]"

    ratings = nx.from_pandas_edgelist(
        df,
        create_using=nx.DiGraph,
        source=col_in,
        target=col_out,
        edge_attr=[col_rating]
    )
    
    F = {}
    G = {}

    for val_in, val_out in tqdm(ratings.edges(), 'Inicializando estructuras'):
        F[val_in] = 1
        G[val_out] = 1
        ratings[val_in][val_out]['R'] = 1

    for i in tqdm(range(n_iteraciones), 'Realizando iteraciones F/G/R'):
        for val_out in G:
            s = 0
            n = 0
            for val_in in ratings.predecessors(val_out):
                s += ratings[val_in][val_out]['R'] * ratings[val_in][val_out][col_rating]
                n += 1
            G[val_out] = s / n
            assert -1 <= G[val_out] <= 1

        for val_in, val_out in ratings.edges():
            R_new = (gamma_1 * F[val_in] + gamma_2 * (1 - (abs(ratings[val_in][val_out][col_rating] - G[val_out]) / 2))) / (gamma_1 + gamma_2)
            ratings[val_in][val_out]['R'] = R_new
            assert 0 <= R_new <= 1

        for val_in in F:
            s = 0
            for val_out in ratings[val_in]:
                s += ratings[val_in][val_out]['R']
            F[val_in] = s / len(ratings[val_in])
            assert 0 <= F[val_in] <= 1

    series_F = df[col_in].apply(lambda x: F[x])
    series_G = df[col_out].apply(lambda x: G[x])
    series_R = df[[col_in, col_out]].apply(lambda x: ratings[x[col_in]][x[col_out]]['R'], axis=1)

    return series_F, series_G, series_R

In [ ]:
series = rev2(
    df_notas,
    'materia_id',
    'Padron',
    'materia_nota_norm',
    n_iteraciones=20,
    gamma_1=0.5,
    gamma_2=0.5,
)

In [ ]:
df_notas['F'] = series[0]
df_notas['G'] = series[1]
df_notas['R'] = series[2]

## Fairness

Bajo este contexto, una materia injusta puede dar nota baja a "buenos estudiantes" y nota alta a "malos estudiantes".

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] > 10)]
        [['materia_id', 'materia_nombre', 'F']]
        .drop_duplicates()
        .sort_values('F', ascending=True)
        .head(5)
)

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] > 30)]
        [['materia_id', 'materia_nombre', 'F']]
        .drop_duplicates()
        .sort_values('F', ascending=False)
        .head(5)
)

## Goodness

Nota que una materia "confiable" le podría dar a tal estudiante. No es lo mismo que el promedio

In [ ]:
df_notas['G_norm'] = df_notas['G'] * 3 + 7
(
    df_notas[(df_notas['cant_materias'] > 10)]
        [['Padron', 'G_norm', 'nota_promedio']]
        .drop_duplicates()
        .sort_values('G_norm', ascending=True)
        .head(5)
)

In [ ]:
(
    df_notas[(df_notas['cant_materias'] > 10)]
        [['Padron', 'G_norm', 'nota_promedio']]
        .drop_duplicates()
        .sort_values('G_norm', ascending=False)
        .head(5)
)

## Reliability

Las notas más esperables y las más injustas

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] > 30)]
        [['Padron', 'materia_id', 'materia_nombre', 'materia_nota', 'R', 'G_norm']]
        .drop_duplicates()
        .sort_values('R', ascending=True)
        .head(15)
)

In [ ]:
(
    df_notas[(df_notas['cant_padrones'] > 30)]
        [['Padron', 'materia_id', 'materia_nombre', 'materia_nota', 'R', 'G_norm']]
        .drop_duplicates()
        .sort_values('R', ascending=False)
        .head(15)
)